In [2]:
import sys
from platform import python_version
print (f"current platform python version: {python_version()}")

current platform python version: 3.11.6


In [3]:
#!{sys.executable} -m pip install --upgrade --user kfp==2.4.0 kfp-kubernetes==1.0.0 kfp-server-api==2.0.5

In [4]:
!{sys.executable} -m pip list | grep kfp

kfp                           2.4.0
kfp-kubernetes                1.0.0
kfp-pipeline-spec             0.2.2
kfp-server-api                2.0.5


In [5]:
from kfp.dsl.pipeline_task import PipelineTask

def set_res_limit(task: PipelineTask, mem_req="200Mi", cpu_req="2000m", mem_lim="4000Mi", cpu_lim='4000m') -> PipelineTask:
    """set the resource limit for cpu and memory, no cpu and memory requirement sofar.
    should the limit is set to small, the Task Pod would be stopped by kubernetes with OOMKilled status.
    
    Args:
        task(PipelineTask): the KFP PipelineTask which need to be set the cpu and memory limits
        cpu_limit(str): the str representation of cpu limit e.g. '1' as one cpu time, '0.5' as 1/2 cpu time
        mem_limit(str): the str representation of memory limit e.g. '500M' for 500MB RAM
        
    Return:
        (PipelineTask): the PipelineTask with the desired limitations set
    """
    # return task.set_cpu_limit('1').set_memory_limit('500M')
    return task.set_cpu_request(cpu_req)\
            .set_cpu_limit(cpu_lim)\
            .set_memory_request(mem_req)\
            .set_memory_limit(mem_lim)

### Simple pipeline
* hello world v2 https://www.kubeflow.org/docs/components/pipelines/v2/hello-world/

In [10]:
from dataclasses import dataclass

@dataclass
class Settings():
    # base_image='python:3.10.13-bullseye' 
    base_image='python:3.11.7-bullseye'

settings = Settings()

In [20]:
from kfp import dsl

@dsl.component(
    base_image=settings.base_image,
)
def say_hello(name: str) -> str:
    hello_text = f"Hello, {name}!"
    print(hello_text)
    return hello_text

@dsl.pipeline(
    name='Helloworld pipeline',
    description='An example kfp v2 simple pipeline'
)
def hello_pipeline(recipient: str) -> str:
    hello_task = say_hello(name=recipient)
    # set resource limit for the task pod 200Mi, 1CPU
    hello_task = set_res_limit(task=hello_task, mem_req="200Mi", cpu_req="1000m", mem_lim="500Mi", cpu_lim='1000m')
    return hello_task.output

my_pipeline = hello_pipeline

In [21]:
from kfp import compiler
import os

my_pipeline_file_name = "hello_pipeline_v2"
pipeline_path_dir = "./compiled"

if not os.path.exists(pipeline_path_dir):
    os.makedirs(pipeline_path_dir)

compiler.Compiler().compile(
    pipeline_func=my_pipeline,
    package_path=f"{pipeline_path_dir}/{my_pipeline_file_name}.yaml"
)

In [9]:
from kfp.client import Client
import warnings

'''suppress kfp v2 client FutureWarning
https://stackoverflow.com/questions/14463277/how-to-disable-python-warnings/14463362#14463362
'''
with warnings.catch_warnings(action="ignore", category=FutureWarning):
    # kubeflow pipeline poddefault is passing the credential to the client
    client = Client()

NAMESPACE = client.get_user_namespace()
EXPERIMENT_NAME = "demo"
print(NAMESPACE)

kubeflow-kindfor
